In [ ]:
!unzip -q /content/drive/MyDrive/Multiome_Cell/data/multimodal-single-cell-as-sparse-matrix.zip

In [ ]:
!pip install -q keras_tuner
import gc
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.utils import plot_model
import keras_tuner
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import GroupKFold,KFold
import lightgbm as lgb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import scipy
from sklearn.decomposition import PCA, TruncatedSVD
from tqdm.notebook import tqdm
import joblib

In [ ]:
def seed(seed=0):
  np.random.seed(seed)  # Numpy module.
  random.seed(seed)
  tf.random.set_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed) 
  os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed(19990829)

In [ ]:
train_inputs = scipy.sparse.load_npz('/content/train_cite_inputs_values.sparse.npz')
train_inputs = np.array(train_inputs.todense())
test_inputs = scipy.sparse.load_npz('/content/test_cite_inputs_values.sparse.npz')
test_inputs = np.array(test_inputs.todense())
train_targets = scipy.sparse.load_npz('/content/train_cite_targets_values.sparse.npz')
train_targets = np.array(train_targets.todense())
cols = np.load('/content/train_cite_inputs_idxcol.npz',allow_pickle=True)['columns']
target_cols = np.load('/content/train_cite_targets_idxcol.npz',allow_pickle=True)['columns']

In [ ]:
## 分析特征中数值为0的占比
analysis_1_train = (train_inputs == 0).sum(axis=0) / len(train_inputs)
analysis_1_test = (test_inputs == 0).sum(axis=0) / len(test_inputs)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(analysis_1_train)

In [ ]:
plt.hist(analysis_1_test)

In [ ]:
not_constant_cols = list(set([k for k,v in enumerate(train_inputs.std(axis=0) == 0) if v == False] + [k for k,v in enumerate(test_inputs.std(axis=0) == 0) if v == False]))
not_constant_cols_2 = list(set([k for k,v in enumerate(analysis_1_train >= 0.999) if v == False] + [k for k,v in enumerate(analysis_1_test >= 0.999) if v == False]))
# not_constant_cols = not_constant_cols_2

# important_cols = []
# for y_col in target_cols:
#     important_cols += [i for i in range(len(cols)) if y_col in cols[i]]
# not_constant_cols = list(set([k for k,v in enumerate(train_inputs.std(axis=0) == 0) if v == False] + [k for k,v in enumerate(test_inputs.std(axis=0) == 0) if v == False]))
important_cols_tmp = [
 'ENSG00000135218_CD36',
 'ENSG00000010278_CD9',
 'ENSG00000204287_HLA-DRA',
 'ENSG00000117091_CD48',
 'ENSG00000004468_CD38',
 'ENSG00000173762_CD7',
 'ENSG00000137101_CD72',
 'ENSG00000019582_CD74',
 'ENSG00000169442_CD52',
 'ENSG00000170458_CD14',
 'ENSG00000272398_CD24',
 'ENSG00000026508_CD44',
 'ENSG00000114013_CD86',
 'ENSG00000174059_CD34',
 'ENSG00000139193_CD27',
 'ENSG00000105383_CD33',
 'ENSG00000085117_CD82',
 'ENSG00000177455_CD19',
 'ENSG00000002586_CD99',
 'ENSG00000196126_HLA-DRB1',
 'ENSG00000135404_CD63',
 'ENSG00000012124_CD22',
 'ENSG00000134061_CD180',
 'ENSG00000105369_CD79A',
 'ENSG00000116824_CD2',
 'ENSG00000010610_CD4',
 'ENSG00000139187_KLRG1',
 'ENSG00000204592_HLA-E',
 'ENSG00000090470_PDCD7',
 'ENSG00000206531_CD200R1L',
'ENSG00000166710_B2M',
 'ENSG00000198034_RPS4X',
 'ENSG00000188404_SELL',
 'ENSG00000130303_BST2',
 'ENSG00000128040_SPINK2',
 'ENSG00000206503_HLA-A',
 'ENSG00000108107_RPL28',
 'ENSG00000143226_FCGR2A',
 'ENSG00000133112_TPT1',
 'ENSG00000166091_CMTM5',
 'ENSG00000026025_VIM',
 'ENSG00000205542_TMSB4X',
 'ENSG00000109099_PMP22',
 'ENSG00000145425_RPS3A',
 'ENSG00000172247_C1QTNF4',
 'ENSG00000072274_TFRC',
 'ENSG00000234745_HLA-B',
 'ENSG00000075340_ADD2',
 'ENSG00000119865_CNRIP1',
 'ENSG00000198938_MT-CO3',
 'ENSG00000135046_ANXA1',
 'ENSG00000235169_SMIM1',
 'ENSG00000101200_AVP',
 'ENSG00000167996_FTH1',
 'ENSG00000163565_IFI16',
 'ENSG00000117450_PRDX1',
 'ENSG00000124570_SERPINB6',
 'ENSG00000112077_RHAG',
 'ENSG00000051523_CYBA',
 'ENSG00000107130_NCS1',
 'ENSG00000055118_KCNH2',
 'ENSG00000029534_ANK1',
 'ENSG00000169567_HINT1',
 'ENSG00000142089_IFITM3',
 'ENSG00000139278_GLIPR1',
 'ENSG00000142227_EMP3',
 'ENSG00000076662_ICAM3',
 'ENSG00000143627_PKLR',
 'ENSG00000130755_GMFG',
 'ENSG00000160593_JAML',
 'ENSG00000095932_SMIM24',
 'ENSG00000197956_S100A6',
 'ENSG00000171476_HOPX',
 'ENSG00000116675_DNAJC6',
 'ENSG00000100448_CTSG',
 'ENSG00000100368_CSF2RB',
 'ENSG00000047648_ARHGAP6',
 'ENSG00000198918_RPL39',
 'ENSG00000196154_S100A4',
 'ENSG00000233968_AL157895.1',
 'ENSG00000137642_SORL1',
 'ENSG00000133816_MICAL2',
 'ENSG00000130208_APOC1',
 'ENSG00000105610_KLF1'
]


important_cols = [i for i in range(len(cols)) if cols[i] in important_cols_tmp]

print(len(not_constant_cols))
print(len(important_cols))

In [ ]:
corr_importance = joblib.load('/content/drive/MyDrive/Multiome_Cell/data/result/corr_importance.pkl')
corr_importance = pd.DataFrame(corr_importance)
corr_importance = corr_importance.T
corr_important_feature = corr_importance[corr_importance > 0.5].sum(axis=1)[corr_importance[corr_importance > 0.5].sum(axis=1) !=0 ].index

In [ ]:
corr_important_feature

In [ ]:
important_cols = list(set(important_cols + list(corr_important_feature)))

In [ ]:
train_not_constant = train_inputs[:,not_constant_cols]
train_important = train_inputs[:,important_cols]
test_not_constant = test_inputs[:,not_constant_cols]
test_important = test_inputs[:,important_cols]
train_test = np.concatenate([train_not_constant,test_not_constant],axis=0)
train_test_important = np.concatenate([train_important,test_important],axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
train_test_important = StandardScaler().fit_transform(train_test_important)
pca_cite = TruncatedSVD(n_components=64, random_state=42)
inputs_pca = pca_cite.fit_transform(train_test)
train_inputs = np.concatenate([inputs_pca[:len(train_inputs)],train_test_important[:len(train_inputs)]],axis=1)
test_inputs = np.concatenate([inputs_pca[len(train_inputs):],train_test_important[len(train_inputs):]],axis=1)

In [ ]:
train_inputs.shape

In [ ]:
train_important.shape[1]

In [ ]:
train_targets  -= train_targets.mean(axis=1).reshape(-1,1)
train_targets /= train_targets.std(axis=1).reshape(-1,1)

In [ ]:
n_splits = 9
meta_df = pd.read_parquet('/content/metadata.parquet')
meta_df = meta_df[meta_df.technology == 'citeseq']

conditions = [
    meta_df['donor'].eq(27678) & meta_df['day'].eq(2),
    meta_df['donor'].eq(27678) & meta_df['day'].eq(3),
    meta_df['donor'].eq(27678) & meta_df['day'].eq(4),
    meta_df['donor'].eq(27678) & meta_df['day'].eq(7),
    meta_df['donor'].eq(13176) & meta_df['day'].eq(2),
    meta_df['donor'].eq(13176) & meta_df['day'].eq(3),
    meta_df['donor'].eq(13176) & meta_df['day'].eq(4),
    meta_df['donor'].eq(13176) & meta_df['day'].eq(7),
    meta_df['donor'].eq(31800) & meta_df['day'].eq(2),
    meta_df['donor'].eq(31800) & meta_df['day'].eq(3),
    meta_df['donor'].eq(31800) & meta_df['day'].eq(4),
    meta_df['donor'].eq(31800) & meta_df['day'].eq(7),
    meta_df['donor'].eq(32606) & meta_df['day'].eq(2),
    meta_df['donor'].eq(32606) & meta_df['day'].eq(3),
    meta_df['donor'].eq(32606) & meta_df['day'].eq(4),
    meta_df['donor'].eq(32606) & meta_df['day'].eq(7)
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

# create a new column and use np.select to assign values to it using our lists as arguments
meta_df['group'] = np.select(conditions, values)

In [ ]:
cite_index_dict = {}
count = 0
for i in tqdm(np.load('/content/train_cite_inputs_idxcol.npz',allow_pickle=True)['index']):
  cite_index_dict[i] = count
  count += 1
meta_df['train_test_index'] = meta_df['cell_id'].map(cite_index_dict)
meta_df['train_test_index'] = meta_df['cell_id'].map(cite_index_dict)
meta_df_group = meta_df[~meta_df['train_test_index'].isna()]
meta_df_group.index = meta_df_group['train_test_index']
kf = GroupKFold(n_splits=n_splits)
for index,(train_index , test_index) in enumerate(kf.split(meta_df_group,meta_df_group,meta_df_group['group'])):
  print(meta_df_group.loc[train_index,'group'].unique(),meta_df_group.loc[test_index,'group'].unique())

In [ ]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

def negative_correlation_loss(y_true, y_pred):
    """Negative correlation loss function for Keras

    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1
    
    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """
    my = K.mean(tf.convert_to_tensor(y_pred), axis=1)
    my = tf.tile(tf.expand_dims(my, axis=1), (1, y_true.shape[1]))
    ym = y_pred - my
    r_num = K.sum(tf.multiply(y_true, ym), axis=1)
    r_den = tf.sqrt(K.sum(K.square(ym), axis=1) * float(y_true.shape[-1]))
    r = tf.reduce_mean(r_num / r_den)
    return - r

In [ ]:
LR_START = 0.01
BATCH_SIZE = 256



def my_model(hp, n_inputs = train_inputs.shape[1]):
    """Sequential neural network
    
    Returns a compiled instance of tensorflow.keras.models.Model.
    """
    activation = 'swish'


    reg1 = hp.Float("reg1", min_value=1e-8, max_value=1e-4, sampling="log")
    reg2 = hp.Float("reg2", min_value=1e-10, max_value=1e-5, sampling="log")

    inputs = Input(shape=(n_inputs, ))

    # attention_probs = Dense(n_inputs, activation='softmax', name='attention_vec')(inputs)
    # attention_mul =  tf.multiply(inputs, attention_probs)

    x0 = tf.keras.layers.BatchNormalization()(inputs)
    x0 = tf.keras.layers.Dropout(0.1)(x0)
    x0 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation = activation,
             )(x0)

    x1 = tf.keras.layers.BatchNormalization()(x0)
    x1 = tf.keras.layers.Dropout(0.1)(x1)
    x1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation = activation,
             )(x1)

    x2 = tf.keras.layers.BatchNormalization()(x1)
    x2 = tf.keras.layers.Dropout(0.1)(x2)
    x2 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation = activation,
             )(x2)

    x3 = tf.keras.layers.BatchNormalization()(x2)
    x3 = tf.keras.layers.Dropout(0.1)(x3)
    x3 = Dense(128, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation = activation,
             )(x3)
    
    
    inputs_important = inputs[:,-train_important.shape[1]:]
    v = tf.random_uniform_initializer(minval=-0.001, maxval=0.001, seed=19990829)((inputs_important.shape[1],300))
    inter_part1 = K.pow(tf.matmul(inputs_important, v), 2)
    inter_part2 = tf.matmul(K.pow(inputs_important, 2), K.pow(v, 2))
    pair_interactions = tf.subtract(inter_part1, inter_part2)
    
     

    x = Concatenate()([x0,x1,x2,x3,pair_interactions])

    # x = tf.keras.layers.BatchNormalization()(x)
    x = Dense(140, kernel_regularizer=tf.keras.regularizers.l2(reg2),
             )(x)

    
    regressor = Model(inputs, x)
    regressor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR_START),
                      metrics=['mse',negative_correlation_loss],
                      loss=negative_correlation_loss
                     )
    
    return regressor

In [ ]:
best_hp = keras_tuner.HyperParameters()
best_hp.values = {
                'reg1': 6.89e-6, #8e-6,
                'reg2': 0, #2e-6,
                'units1': 256,
                'units2': 256,
                'units3': 256,
                'units4': 128
                } 
# 'reg1': 6.89e-6, #8e-6,

In [ ]:
%%time
VERBOSE = 1 # set to 2 for more output
EPOCHS = 1000
N_SPLITS = 9


kf = GroupKFold(n_splits = N_SPLITS)
score_list = []
va_pred = []
for fold,(train_index , test_index) in enumerate(kf.split(meta_df_group,meta_df_group,meta_df_group['group'])):
    model = None
    gc.collect()

    X_tr , y_tr = train_inputs[train_index] , train_targets[train_index] 
    X_va ,  y_va  = train_inputs[test_index] , train_targets[test_index]
    

    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                           patience=4, verbose=VERBOSE)
    es = EarlyStopping(monitor="val_loss",
                       patience=12, 
                       verbose=0,
                       mode="min", 
                       restore_best_weights=True)
    callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

    # Construct and compile the model
    model = my_model(best_hp, X_tr.shape[1])

    # Train the model
    history = model.fit(X_tr, y_tr, 
                        validation_data=(X_va, y_va), 
                        epochs=EPOCHS,
                        verbose=VERBOSE,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        callbacks=callbacks)
    del X_tr, y_tr
    
    model.save(f"/content/drive/MyDrive/Multiome_Cell/data/model/tensorflow_dnn/tf_cite_fm_model_v2_{fold}")
    history = history.history
    callbacks, lr = None, None
    
    # We validate the model
    y_va_pred = model.predict(X_va, batch_size=len(X_va))
    va_pred.append(y_va_pred)
    corrscore = correlation_score(y_va, y_va_pred)

    print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
    del es, X_va#, y_va, y_va_pred
    score_list.append(corrscore)

# Show overall score
print(f"corr = {np.array(score_list).mean():.5f}")

In [ ]:
print(score_list , f"corr = {np.array(score_list).mean():.5f}")

In [ ]:
def predict():
  preds = np.zeros((len(test_inputs), 140), dtype=np.float32)
  for fold in range(N_SPLITS):
      print(f"Predicting with fold {fold}")
      model = load_model(f"/content/drive/MyDrive/Multiome_Cell/data/model/tensorflow_dnn/tf_cite_fm_model_v2_{fold}",custom_objects={'negative_correlation_loss': negative_correlation_loss})
      preds += model.predict(test_inputs)

  test_pred = preds / N_SPLITS

  submission = pd.read_csv('/content/drive/MyDrive/Multiome_Cell/data/result/multi_dnn_model_v1_cite.csv',
                            index_col='row_id', squeeze=True)


  submission.iloc[:len(test_pred.ravel())] = test_pred.ravel()
  assert not submission.isna().any()
  submission.to_csv('/content/drive/MyDrive/Multiome_Cell/data/result/tf_cite_model_submission_v2.csv')
  display(submission)

In [ ]:
predict()